# Optimización Bayesiana de Hiperparámetros - YOLOv11 Pothole Detection

Este notebook utiliza Optuna para realizar optimización bayesiana y encontrar los mejores hiperparámetros para entrenar YOLOv11 en la detección de baches.

## Hardware: RTX 5090 (32GB VRAM)
## Task: Instance Segmentation

In [ ]:
import os
import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances
from ultralytics import YOLO
import torch
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
from datetime import datetime

# Configuración de estilo
sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (12, 8)

In [ ]:
# Verificar GPU disponible
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM Total: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    print(f"VRAM Available: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")

In [ ]:
# Configuración de rutas
BASE_DIR = Path('..')
CONFIG_DIR = BASE_DIR / 'config'
DATASET_CONFIG = CONFIG_DIR / 'pothole_dataset.yaml'
MODELS_DIR = BASE_DIR / 'models'
OPTIMIZATION_DIR = MODELS_DIR / 'optimization'
OPTIMIZATION_DIR.mkdir(parents=True, exist_ok=True)

print(f"Dataset Config: {DATASET_CONFIG}")
print(f"Models Directory: {MODELS_DIR}")
print(f"Optimization Directory: {OPTIMIZATION_DIR}")

In [ ]:
# Configuración de la optimización
OPTIMIZATION_CONFIG = {
    'n_trials': 50,  # Número de trials para la optimización bayesiana
    'n_epochs_per_trial': 30,  # Epochs por trial (reducido para optimización)
    'imgsz': 640,  # Tamaño de imagen base
    'model_variant': 'yolo11n-seg.pt',  # Usar nano para optimización rápida
    'device': 0,  # GPU 0
    'workers': 8,  # Workers para dataloader
    'patience': 10,  # Early stopping patience
}

print("Configuración de optimización:")
for key, value in OPTIMIZATION_CONFIG.items():
    print(f"  {key}: {value}")

## Definición de la Función Objetivo

Esta función será optimizada por Optuna. Retorna la métrica mAP50-95 (box) como objetivo a maximizar.

In [ ]:
def objective(trial):
    """
    Función objetivo para Optuna.
    
    Hiperparámetros a optimizar:
    - Learning rate inicial (lr0)
    - Learning rate final (lrf)
    - Momentum
    - Weight decay
    - Warmup epochs
    - Batch size
    - Image size
    - Mosaic augmentation
    - Mixup augmentation
    - HSV augmentations (Hue, Saturation, Value)
    - Flip augmentation probability
    - Scale augmentation
    - Translate augmentation
    - Box loss gain
    - Cls loss gain
    - DFL loss gain
    """
    
    # Hiperparámetros de optimización
    lr0 = trial.suggest_float('lr0', 1e-5, 1e-2, log=True)
    lrf = trial.suggest_float('lrf', 0.01, 0.2)
    momentum = trial.suggest_float('momentum', 0.8, 0.99)
    weight_decay = trial.suggest_float('weight_decay', 1e-5, 1e-3, log=True)
    warmup_epochs = trial.suggest_int('warmup_epochs', 1, 5)
    
    # Batch size y image size (aprovechando la RTX 5090)
    batch_size = trial.suggest_categorical('batch', [16, 24, 32, 40])
    imgsz = trial.suggest_categorical('imgsz', [640, 768, 896])
    
    # Data augmentation
    mosaic = trial.suggest_float('mosaic', 0.5, 1.0)
    mixup = trial.suggest_float('mixup', 0.0, 0.3)
    hsv_h = trial.suggest_float('hsv_h', 0.0, 0.1)
    hsv_s = trial.suggest_float('hsv_s', 0.0, 0.9)
    hsv_v = trial.suggest_float('hsv_v', 0.0, 0.9)
    flipud = trial.suggest_float('flipud', 0.0, 0.5)
    fliplr = trial.suggest_float('fliplr', 0.3, 0.7)
    scale = trial.suggest_float('scale', 0.3, 0.7)
    translate = trial.suggest_float('translate', 0.05, 0.2)
    
    # Loss gains
    box_gain = trial.suggest_float('box', 5.0, 10.0)
    cls_gain = trial.suggest_float('cls', 0.3, 1.0)
    dfl_gain = trial.suggest_float('dfl', 1.0, 2.5)
    
    # Optimizer
    optimizer = trial.suggest_categorical('optimizer', ['SGD', 'Adam', 'AdamW'])
    
    # Nombre del trial
    trial_name = f"trial_{trial.number:03d}"
    trial_dir = OPTIMIZATION_DIR / trial_name
    
    try:
        # Cargar modelo
        model = YOLO(OPTIMIZATION_CONFIG['model_variant'])
        
        # Entrenar con los hiperparámetros sugeridos
        results = model.train(
            data=str(DATASET_CONFIG),
            epochs=OPTIMIZATION_CONFIG['n_epochs_per_trial'],
            imgsz=imgsz,
            batch=batch_size,
            device=OPTIMIZATION_CONFIG['device'],
            workers=OPTIMIZATION_CONFIG['workers'],
            patience=OPTIMIZATION_CONFIG['patience'],
            project=str(OPTIMIZATION_DIR),
            name=trial_name,
            exist_ok=True,
            pretrained=True,
            
            # Hiperparámetros optimizados
            lr0=lr0,
            lrf=lrf,
            momentum=momentum,
            weight_decay=weight_decay,
            warmup_epochs=warmup_epochs,
            optimizer=optimizer,
            
            # Augmentations
            mosaic=mosaic,
            mixup=mixup,
            hsv_h=hsv_h,
            hsv_s=hsv_s,
            hsv_v=hsv_v,
            flipud=flipud,
            fliplr=fliplr,
            scale=scale,
            translate=translate,
            
            # Loss gains
            box=box_gain,
            cls=cls_gain,
            dfl=dfl_gain,
            
            # Otras configuraciones
            verbose=False,
            plots=False,
            save=True,
            save_period=-1,  # Solo guardar el mejor modelo
        )
        
        # Obtener métricas de validación
        # Para segmentación, usamos metrics/mAP50-95(M) que es la métrica de máscara
        map_score = results.results_dict.get('metrics/mAP50-95(M)', 0.0)
        
        # Guardar información del trial
        trial_info = {
            'trial_number': trial.number,
            'mAP50-95': map_score,
            'hyperparameters': trial.params,
            'timestamp': datetime.now().isoformat()
        }
        
        with open(trial_dir / 'trial_info.json', 'w') as f:
            json.dump(trial_info, f, indent=2)
        
        # Limpiar memoria
        del model
        torch.cuda.empty_cache()
        
        return map_score
        
    except Exception as e:
        print(f"Error en trial {trial.number}: {str(e)}")
        # Limpiar memoria en caso de error
        torch.cuda.empty_cache()
        return 0.0

## Ejecutar Optimización Bayesiana

In [ ]:
# Crear estudio de Optuna
study_name = f"yolo11_pothole_optimization_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
storage_name = f"sqlite:///{OPTIMIZATION_DIR}/{study_name}.db"

study = optuna.create_study(
    study_name=study_name,
    direction='maximize',  # Maximizar mAP
    storage=storage_name,
    load_if_exists=True,
    sampler=optuna.samplers.TPESampler(seed=42),  # Tree-structured Parzen Estimator
    pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=10)
)

print(f"Estudio creado: {study_name}")
print(f"Storage: {storage_name}")
print(f"\nIniciando optimización bayesiana con {OPTIMIZATION_CONFIG['n_trials']} trials...")

In [ ]:
# Ejecutar optimización
study.optimize(
    objective, 
    n_trials=OPTIMIZATION_CONFIG['n_trials'],
    show_progress_bar=True,
    n_jobs=1  # Usar 1 job debido al uso de GPU
)

print("\n" + "="*80)
print("OPTIMIZACIÓN COMPLETADA")
print("="*80)

## Análisis de Resultados

In [ ]:
# Mejores hiperparámetros
print("\nMEJORES HIPERPARÁMETROS ENCONTRADOS:")
print("="*80)
print(f"Mejor mAP50-95: {study.best_value:.4f}")
print(f"\nHiperparámetros:")
for param, value in study.best_params.items():
    print(f"  {param}: {value}")

# Guardar mejores hiperparámetros
best_params_file = OPTIMIZATION_DIR / 'best_hyperparameters.json'
with open(best_params_file, 'w') as f:
    json.dump({
        'best_value': study.best_value,
        'best_params': study.best_params,
        'best_trial': study.best_trial.number,
        'study_name': study_name,
        'timestamp': datetime.now().isoformat()
    }, f, indent=2)

print(f"\nMejores hiperparámetros guardados en: {best_params_file}")

In [ ]:
# Convertir trials a DataFrame para análisis
trials_df = study.trials_dataframe()
trials_df.to_csv(OPTIMIZATION_DIR / 'optimization_trials.csv', index=False)

print(f"Total de trials completados: {len(trials_df)}")
print(f"\nEstadísticas de mAP50-95:")
print(trials_df['value'].describe())

# Mostrar top 10 trials
print("\nTop 10 Trials:")
top_trials = trials_df.nlargest(10, 'value')[['number', 'value']]
print(top_trials)

## Visualizaciones

In [ ]:
# Historia de optimización
fig1 = plot_optimization_history(study)
fig1.update_layout(
    title='Historia de Optimización Bayesiana',
    xaxis_title='Trial',
    yaxis_title='mAP50-95 (Mask)',
    width=1200,
    height=600
)
fig1.write_html(OPTIMIZATION_DIR / 'optimization_history.html')
fig1.show()

In [ ]:
# Importancia de hiperparámetros
fig2 = plot_param_importances(study)
fig2.update_layout(
    title='Importancia de Hiperparámetros',
    width=1200,
    height=800
)
fig2.write_html(OPTIMIZATION_DIR / 'param_importances.html')
fig2.show()

In [ ]:
# Distribución de valores por hiperparámetro
from optuna.visualization import plot_slice

fig3 = plot_slice(study)
fig3.update_layout(
    title='Distribución de Hiperparámetros vs mAP',
    width=1400,
    height=1000
)
fig3.write_html(OPTIMIZATION_DIR / 'param_slices.html')
fig3.show()

In [ ]:
# Parallel coordinate plot
from optuna.visualization import plot_parallel_coordinate

fig4 = plot_parallel_coordinate(study)
fig4.update_layout(
    title='Parallel Coordinate Plot',
    width=1400,
    height=800
)
fig4.write_html(OPTIMIZATION_DIR / 'parallel_coordinate.html')
fig4.show()

In [ ]:
# Correlación entre hiperparámetros
plt.figure(figsize=(16, 14))

# Seleccionar columnas numéricas de parámetros
param_cols = [col for col in trials_df.columns if col.startswith('params_')]
correlation_data = trials_df[param_cols + ['value']].copy()

# Renombrar columnas para mejor visualización
correlation_data.columns = [col.replace('params_', '') for col in correlation_data.columns]

# Calcular correlación
correlation_matrix = correlation_data.corr()

# Plot heatmap
sns.heatmap(
    correlation_matrix, 
    annot=True, 
    fmt='.2f', 
    cmap='coolwarm', 
    center=0,
    square=True,
    linewidths=1
)
plt.title('Correlación entre Hiperparámetros y mAP', fontsize=16, pad=20)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig(OPTIMIZATION_DIR / 'correlation_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

## Crear Archivo de Configuración para Entrenamiento Final

In [ ]:
# Crear configuración YAML con los mejores hiperparámetros
training_config = {
    'model': 'yolo11x-seg.pt',  # Usar el modelo más grande para entrenamiento final
    'data': str(DATASET_CONFIG),
    'epochs': 300,  # Más epochs para entrenamiento final
    'patience': 50,
    'device': 0,
    'workers': 8,
    'project': '../models/final_training',
    'name': 'yolo11x_pothole_best',
    
    # Mejores hiperparámetros encontrados
    **study.best_params
}

# Guardar configuración
training_config_file = CONFIG_DIR / 'training_config.yaml'
with open(training_config_file, 'w') as f:
    yaml.dump(training_config, f, default_flow_style=False)

print(f"Configuración de entrenamiento guardada en: {training_config_file}")
print("\nContenido:")
print(yaml.dump(training_config, default_flow_style=False))

## Resumen Final

In [ ]:
print("\n" + "="*80)
print("RESUMEN DE OPTIMIZACIÓN BAYESIANA")
print("="*80)
print(f"\nTotal de trials ejecutados: {len(study.trials)}")
print(f"Mejor mAP50-95 alcanzado: {study.best_value:.4f}")
print(f"Trial con mejor resultado: {study.best_trial.number}")
print(f"\nArchivos generados:")
print(f"  - Mejores hiperparámetros: {best_params_file}")
print(f"  - Configuración de entrenamiento: {training_config_file}")
print(f"  - Base de datos de trials: {storage_name}")
print(f"  - CSV de trials: {OPTIMIZATION_DIR / 'optimization_trials.csv'}")
print(f"  - Visualizaciones: {OPTIMIZATION_DIR}/*.html")
print(f"\n{'='*80}")
print("¡Optimización completada! Procede al notebook de entrenamiento final.")
print("="*80)